### Import Required Libraries and Set Up Environment Variables

In [18]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [19]:
# Set environment variables from the .env in the local environment
load_dotenv("/users/shephalidubey/Documents/keys.env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


In [20]:
type(nyt_api_key)

str

In [21]:
type(tmdb_api_key)

str

### Access the New York Times API

In [22]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"


In [23]:
# Build URL
query_url = (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}")

In [24]:
####get the response. If it comes to 200 , you are good, otherwise stop as something is not right
response = requests.get(query_url)
response

<Response [200]>

In [25]:
# added to Preview data for next exercise
#reviews = response.json()
#print(json.dumps(reviews,indent=4))

In [26]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
    # create query with a page number
    # API results show 10 articles at a time

#don't need counter, can use range instead
for page in range(0,20):           
    query_url2 = f"{query_url}&page={page}"        
                    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url2).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list

        # loop through the reviews["response"]["docs"] and append each review to the list
    try:
        for review in reviews['response']['docs']:
            reviews_list.append(review)
            # below block of code was not working because i forgot to change the word doc below to review. 
            # In Dave's exercise he used "for doc in reviews", whereas i used , "for review in reviews".
            # also, i don't need to add below block of code because i need the whole response & docs list, not restricted to only below columns
            # ({
            #     'byline': doc['byline']['original'],
            #     'source': doc['source'],
            #     'headline': doc['headline']['main'],
            #     'snippet':doc['snippet'],
            #     'web_url':doc['web_url']
            # })
      
        # Print the page that was just retrieved
        print(f'Checked page {page}')

        # Print the page number that had no results then break from the loop
    except:
        print(f'Checked page{page}not found')
        break
        

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [60]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [61]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
results_df = pd.json_normalize(reviews_list)
results_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [62]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

results_df['title'] = results_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
nyt_results_df = results_df.copy()
nyt_results_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [63]:
# # Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string

nyt_results_df['keywords'] = nyt_results_df['keywords'].apply(extract_keywords)
nyt_results_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [76]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

title_list = nyt_results_df['title'].tolist()   
title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [65]:
# Prepare The Movie Database query
url3 = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [66]:
# Create an empty list to store the results
tmdb_movies_list = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
    # Add 1 to the request counter
    
request_counter = 1

# Loop through the titles

for title in title_list:
    if (request_counter%50) == 0:
        # Check if we need to sleep before making a request
        print(f'Application is sleeping as it has reached {request_counter}request counter limit')
        time.sleep(1)
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    requests_response = requests.get(url3 + title + tmdb_key_string).json()
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
    
    try:
        movie_id = requests_response["results"][0]["id"]
    
        # Make a request for the full movie details
        #Per Alberto Follow the string on movie details on TMDB but instead of language after movie id , add TMDB api key this query is not giving the correct fields
        movie_query = (f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}") 
        
        # Execute "GET" request with url
        full_movie_details = requests.get(movie_query).json()
        
        # Extract the genre names into a list
        genre_list =[]
        for genre in full_movie_details["genres"]:
            genre_list.append(genre["name"])
        
        # Extract the spoken_languages' English name into a list
        spoken_languages_list = []
        for spoken_languages in full_movie_details["spoken_languages"]:
            spoken_languages_list.append(spoken_languages['english_name'])
        
        # Extract the production_countries' name into a list
        production_countries_list = []
        for production_countries in full_movie_details["production_countries"]:
            production_countries_list.append(production_countries["name"]) 
        
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list 

        movie_details = {'title': full_movie_details['title'],
                         'original_title': full_movie_details['original_title'],
                         'budget': full_movie_details['budget'],
                         'original_language': full_movie_details['original_language'],
                         'homepage': full_movie_details['homepage'],
                         'overview': full_movie_details['overview'],
                         'popularity': full_movie_details['popularity'],
                         'runtime': full_movie_details['runtime'],
                         'revenue': full_movie_details['revenue'],
                         'release_date': full_movie_details['release_date'],
                         'vote_average': full_movie_details['vote_average'],
                         'vote_count': full_movie-details['vote_count'],
                         "genres": genre_list,
                         "spoken_languages": spoken_languages,
                         "production_countries": production_countries}
        tmdb_movies_list.append(movie_details)
        
        #Print out the title that was found

        print(tmdb_movies_list)

        #Print the page number that had no results then break from the loop
    except:
        print(f'not found')
        break
        

not found


In [67]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(tmdb_movies_list[:5], indent=4))

[]


In [68]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(tmdb_movies_list[:5], indent=4))

[]


In [69]:
# Convert the results to a DataFrame
tmdb_results_df = pd.json_normalize(tmdb_movies_list)
tmdb_results_df


""


### Merge and Clean the Data for Export

In [70]:
# Merge the New York Times reviews and TMDB DataFrames on title
# merge_df=pd.merge(results_df,tmdb_results_df,on =['title'])
# merge_df.head()

merge_df=pd.merge(tmdb_results_df,nyt_results_df,on ='title')    
merge_df.head()



KeyError: 'title'

In [71]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres","spoken_languages","production_countries"]

# Create a list of characters to remove
characters_to_remove = ["'","[","]"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merge_df[column] = merge_df[column].astype(str)

    # Loop through characters to remove
    for character in characters_to_remove:
        merge_df[column] = merge_df[column].str.replace(character, "")

# Display the fixed DataFrame
merge_df.head()

NameError: name 'merge_df' is not defined

In [72]:
# Drop "byline.person" column
pd.drop(columns='byline.person')

AttributeError: module 'pandas' has no attribute 'drop'

In [73]:
# Delete duplicate rows and reset index
merge_df = merge_df.drop_duplicates().reset_index(drop=True)
merge_df

NameError: name 'merge_df' is not defined

In [74]:
# Export data to CSV without the index


In [75]:
merge_df.csv('movies_cleaned_data.csv',index=False)

NameError: name 'merge_df' is not defined